# Example Notebook to load multiple dataset from different STAC catalogs

### import packages

In [6]:
# Created by Etiënne Kras, 24-10-2024, using geo_env

import os
import pandas as pd
import pystac_client
import xarray as xr
import rioxarray as rio

from copy import deepcopy
from typing import List, Dict

### Functions

In [7]:
# define function(s)

# function to put items in dataframe
def items_to_dataframe(items: List[Dict]) -> pd.DataFrame:
    """STAC items to Pandas dataframe.

    Args:
        items (List[Dict]): _description_

    Returns:
        pd.DataFrame: _description_
    """
    _items = []
    for i in items:
        _i = deepcopy(i)
        # _i['geometry'] = shape(_i['geometry'])
        # ...  # for example, drop some attributes that you're not interested in
        _items.append(_i)
    df = pd.DataFrame(pd.json_normalize(_items))
    # for field in ["properties.datetime"]:
    #     if field in df:
    #         df[field] = pd.to_datetime(df[field])
    # df = df.sort_values("properties.datetime")
    return df

### Open data from first catalog (SLR projections AR6)

In [2]:
# load the CoCliCo STAC catalog
catalog = pystac_client.Client.open(
    "https://storage.googleapis.com/coclico-data-public/coclico/coclico-stac-4oct/catalog.json"
)
# catalog

# list the datasets present in the catalog, we are interested in the slp5 and slp6 sets
list(catalog.get_children())

c:\Users\kras\AppData\Local\mambaforge\envs\geo_env\lib\site-packages\pystac_client\client.py:186: NoConformsTo: Server does not advertise any conformance classes.
  warnings.warn(NoConformsTo())


[<CollectionClient id=ssl>,
 <CollectionClient id=wef>,
 <CollectionClient id=eesl>,
 <CollectionClient id=floodmaps>,
 <CollectionClient id=sc>,
 <CollectionClient id=cbca>,
 <CollectionClient id=cfr>,
 <CollectionClient id=smd>,
 <CollectionClient id=cisi>,
 <CollectionClient id=slp5>,
 <CollectionClient id=slp6>,
 <CollectionClient id=coastal-mask>,
 <CollectionClient id=overture-building-footprints>,
 <CollectionClient id=deltares-delta-dtm>,
 <CollectionClient id=gctr>,
 <CollectionClient id=pp>,
 <CollectionClient id=ceed>,
 <CollectionClient id=shorelinemonitor-shorelines>,
 <CollectionClient id=shorelinemonitor-raw-series>,
 <CollectionClient id=gcts>,
 <CollectionClient id=slp>,
 <CollectionClient id=ss_wc>,
 <CollectionClient id=twl>,
 <CollectionClient id=cet>,
 <CollectionClient id=LAU>,
 <CollectionClient id=NUTS>,
 <CollectionClient id=cfhp>,
 <CollectionClient id=CFHP_LAU_stats>]

In [22]:
# we can now browse the datasets by interacting with the STAC catalog
items = list(catalog.get_child("coastal-mask").get_all_items())

c:\Users\kras\AppData\Local\mambaforge\envs\geo_env\lib\site-packages\pystac_client\collection_client.py:145: FallbackToPystac: Falling back to pystac. This might be slow.
  root._warn_about_fallback("ITEM_SEARCH")


In [8]:
%%time
# read STAC items as Pandas dataframe

# TODO: how to speed up?? 
# select one ens ..

# AR6, takes a while +/- 6 min
ar6_col = catalog.get_child("slp6")
items_ar6 = list(ar6_col.get_items()) # this is slow as we need to list all items
items_ar6_df = items_to_dataframe([i.to_dict() for i in items_ar6])

c:\Users\kras\AppData\Local\mambaforge\envs\geo_env\lib\site-packages\pystac_client\collection_client.py:145: FallbackToPystac: Falling back to pystac. This might be slow.
  root._warn_about_fallback("ITEM_SEARCH")


CPU times: total: 3min 38s
Wall time: 8min 11s


In [12]:
items_ar6_df

,type,stac_version,id,links,bbox,stac_extensions,collection,properties.proj:bbox,properties.proj:shape,properties.proj:epsg,properties.proj:geometry.type,properties.proj:geometry.coordinates,properties.proj:transform,properties.datetime,geometry.type,geometry.coordinates,assets.data.href,assets.data.type,assets.data.roles
0,Feature,1.0.0,ssp=1-26/slr_ens0.0/2020.tif,"[{'rel': 'collection', 'href': '../../../colle...","[-180.5, -90.5, 179.5, 90.5]",[https://stac-extensions.github.io/projection/...,slp6,"[-180.5, -90.5, 179.5, 90.5]","[181, 360]",4326,Polygon,"[[[179.5, -90.5], [179.5, 90.5], [-180.5, 90.5...","[1.0, 0.0, -180.5, 0.0, 1.0, -90.5]",1970-01-01T00:00:00.000002Z,Polygon,"[[[179.5, -90.5], [179.5, 90.5], [-180.5, 90.5...",https://storage.googleapis.com/dgds-data-publi...,image/tiff; application=geotiff; profile=cloud...,[data]
1,Feature,1.0.0,ssp=1-26/slr_ens0.1/2020.tif,"[{'rel': 'collection', 'href': '../../../colle...","[-180.5, -90.5, 179.5, 90.5]",[https://stac-extensions.github.io/projection/...,slp6,"[-180.5, -90.5, 179.5, 90.5]","[181, 360]",4326,Polygon,"[[[179.5, -90.5], [179.5, 90.5], [-180.5, 90.5...","[1.0, 0.0, -180.5, 0.0, 1.0, -90.5]",1970-01-01T00:00:00.000002Z,Polygon,"[[[179.5, -90.5], [179.5, 90.5], [-180.5, 90.5...",https://storage.googleapis.com/dgds-data-publi...,image/tiff; application=geotiff; profile=cloud...,[data]
2,Feature,1.0.0,ssp=1-26/slr_ens0.5/2020.tif,"[{'rel': 'collection', 'href': '../../../colle...","[-180.5, -90.5, 179.5, 90.5]",[https://stac-extensions.github.io/projection/...,slp6,"[-180.5, -90.5, 179.5, 90.5]","[181, 360]",4326,Polygon,"[[[179.5, -90.5], [179.5, 90.5], [-180.5, 90.5...","[1.0, 0.0, -180.5, 0.0, 1.0, -90.5]",1970-01-01T00:00:00.000002Z,Polygon,"[[[179.5, -90.5], [179.5, 90.5], [-180.5, 90.5...",https://storage.googleapis.com/dgds-data-publi...,image/tiff; application=geotiff; profile=cloud...,[data]
3,Feature,1.0.0,ssp=1-26/slr_ens1.0/2020.tif,"[{'rel': 'collection', 'href': '../../../colle...","[-180.5, -90.5, 179.5, 90.5]",[https://stac-extensions.github.io/projection/...,slp6,"[-180.5, -90.5, 179.5, 90.5]","[181, 360]",4326,Polygon,"[[[179.5, -90.5], [179.5, 90.5], [-180.5, 90.5...","[1.0, 0.0, -180.5, 0.0, 1.0, -90.5]",1970-01-01T00:00:00.000002Z,Polygon,"[[[179.5, -90.5], [179.5, 90.5], [-180.5, 90.5...",https://storage.googleapis.com/dgds-data-publi...,image/tiff; application=geotiff; profile=cloud...,[data]
4,Feature,1.0.0,ssp=1-26/slr_ens2.0/2020.tif,"[{'rel': 'collection', 'href': '../../../colle...","[-180.5, -90.5, 179.5, 90.5]",[https://stac-extensions.github.io/projection/...,slp6,"[-180.5, -90.5, 179.5, 90.5]","[181, 360]",4326,Polygon,"[[[179.5, -90.5], [179.5, 90.5], [-180.5, 90.5...","[1.0, 0.0, -180.5, 0.0, 1.0, -90.5]",1970-01-01T00:00:00.000002Z,Polygon,"[[[179.5, -90.5], [179.5, 90.5], [-180.5, 90.5...",https://storage.googleapis.com/dgds-data-publi...,image/tiff; application=geotiff; profile=cloud...,[data]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4489,Feature,1.0.0,ssp=5-85/slr_ens98.0/2150.tif,"[{'rel': 'collection', 'href': '../../../colle...","[-180.5, -90.5, 179.5, 90.5]",[https://stac-extensions.github.io/projection/...,slp6,"[-180.5, -90.5, 179.5, 90.5]","[181, 360]",4326,Polygon,"[[[179.5, -90.5], [179.5, 90.5], [-180.5, 90.5...","[1.0, 0.0, -180.5, 0.0, 1.0, -90.5]",1970-01-01T00:00:00.000002Z,Polygon,"[[[179.5, -90.5], [179.5, 90.5], [-180.5, 90.5...",https://storage.googleapis.com/dgds-data-publi...,image/tiff; application=geotiff; profile=cloud...,[data]
4490,Feature,1.0.0,ssp=5-85/slr_ens99.0/2150.tif,"[{'rel': 'collection', 'href': '../../../colle...","[-180.5, -90.5, 179.5, 90.5]",[https://stac-extensions.github.io/projection/...,slp6,"[-180.5, -90.5, 179.5, 90.5]","[181, 360]",4326,Polygon,"[[[179.5, -90.5], [179.5, 90.5], [-180.5, 90.5...","[1.0, 0.0, -180.5, 0.0, 1.0, -90.5]",1970-01-01T00:00:00.000002Z,Polygon,"[[[179.5, -90.5], [179.5, 90.5], [-180.5, 90.5...",https://storage.goog

In [19]:
sc_col = catalog.get_child("sc")
href = sc_col.assets["data"].href

# Printing the dataset object shows the storm surge level consists of three dimensions.
# Every storm surge level is associated with a certain station, scenario and revisting period.
ds = xr.open_zarr(href)
ds

<xarray.Dataset>
Dimensions:    (nensemble: 7, stations: 518005, nscenarios: 2, time: 2)
Coordinates:
    ensemble   (nensemble) |S3 dask.array<chunksize=(7,), meta=np.ndarray>
    lat        (stations) float64 dask.array<chunksize=(64751,), meta=np.ndarray>
    lon        (stations) float64 dask.array<chunksize=(64751,), meta=np.ndarray>
    scenarios  (nscenarios) |S5 dask.array<chunksize=(2,), meta=np.ndarray>
  * time       (time) int32 2050 2100
Dimensions without coordinates: nensemble, stations, nscenarios
Data variables:
    sc         (nscenarios, stations, time, nensemble) float64 dask.array<chunksize=(1, 64751, 1, 2), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.8
    crs:          4326
    description:  Projections of global shoreline change in view of climate c...
    source:       The procedure to produce the dataset and the findings are d...
    title:        Global shoreline change projections for the year 2050 under...

### Open data from second catalog (future shorelines, world pop & subsidence)

In [20]:
# load the GCA SOTC STAC catalog
catalog2 = pystac_client.Client.open(
    "https://storage.googleapis.com/dgds-data-public/gca/SOTC/gca-stac_sotc/catalog.json"
)
# catalog

# list the datasets present in the catalog, we are interested in the slp5 and slp6 sets
list(catalog2.get_children())

c:\Users\kras\AppData\Local\mambaforge\envs\geo_env\lib\site-packages\pystac_client\client.py:186: NoConformsTo: Server does not advertise any conformance classes.
  warnings.warn(NoConformsTo())


APIError: <?xml version='1.0' encoding='UTF-8'?><Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Details>No such object: dgds-data-public/gca/SOTC/gca-stac_sotc/world_pop/collection.json</Details></Error>

In [ ]:


# shore_mon_fut 
or 
# sc in the coclico catalog..